In [1]:
#!pip install bertviz

In [1]:
from bertviz import head_view


In [3]:
from transformers import BertTokenizer, BertForQuestionAnswering, BertConfig

from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import EMBERT
from ebert.embeddings import load_embedding, MappedEmbedding

2023-04-05 17:37:15 [INFO] modeling_bert: Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2023-04-05 17:37:15 [INFO] modeling_xlnet: Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [4]:
import torch
import torch.nn as nn

In [5]:
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = 'output/monobert-large-msmarco-dbpedia_acc_batch_64_e6_annotated/monobert-large-msmarco-dbpedia_acc_batch_64_e6_fold1_annotated'

# load model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load model

model, model_emb = EMBERT.get_model(pretrained_model_name_or_path = model_path)
reranker =  EMBERT(model = (model, model_emb))
wiki_emb, mapper = reranker.ebert

# reranker =  MonoBERT()
# model, model_emb = reranker.get_model()
# wiki_emb, mapper = reranker.ebert




Loading regular BERT)


2023-04-05 17:37:23 [INFO] modeling_utils: loading configuration file output/monobert-large-msmarco-dbpedia_acc_batch_64_e6_annotated/monobert-large-msmarco-dbpedia_acc_batch_64_e6_fold1_annotated/config.json
2023-04-05 17:37:23 [INFO] modeling_utils: Model config {
  "_name_or_path": "models/monobert-large-msmarco",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pad_token_id": 0,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

2023-04-05 17:37:23 [INFO] modeling_utils: l

Loading EBERT embeddings
MODELTYPE IS  monobert


2023-04-05 17:37:29 [INFO] modeling_utils: Weights from pretrained model not used in BertModel: ['bert.embeddings.position_ids']
2023-04-05 17:37:29 [INFO] tokenization_utils: Model name 'output/monobert-large-msmarco-dbpedia_acc_batch_64_e6_annotated/monobert-large-msmarco-dbpedia_acc_batch_64_e6_fold1_annotated' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc). Assuming 'output/monobert-large-msmarco-dbpedia_acc_batch_64_e6_annotated/monobert-large-msmarco-dbpedia_acc_batch_64_e6_fold1_annotated' is a path or url to a directory containing tokenizer files.
2023-04-05 17:37:29 [INFO] tokenization_utils: load

Loading  resources/wikipedia2vec/wikipedia-20190701/wikipedia2vec_500.pkl  and  mappers/wikipedia2vec-500-cased.monobert-base-cased.linear.npy


In [6]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
model.to(device)
model.eval()
model.zero_grad()

In [12]:
# ret = reranker.tokenize_with_entities('Give me all movies directed by Francis Ford Coppola ENTITY/Francis_Ford_Coppola',
#                                     'Francis Ford Coppola ENTITY/Francis_Ford_Coppola  (/ˈkoʊpələ/; born April 7, 1939) is an American ENTITY/Americans  film director, producer and screenwriter. He was part of the New Hollywood ENTITY/New_Hollywood  wave of filmmaking.After directing The Rain People ENTITY/The_Rain_People  in 1969, he won the Academy Award for Best Original Screenplay ENTITY/Academy_Award_for_Best_Original_Screenplay  as co-writer, with Edmund H. North ENTITY/Edmund_H._North , of Patton ENTITY/Patton_(film)  in 1970.')

# ret = reranker.tokenize_with_entities('Give me all movies directed by Francis Ford Coppola',
#                                       'Rumble Fish is an American 1983 drama film directed by Francis Ford Coppola.')

# ret = reranker.tokenize_with_entities('Give me all movies directed by Francis Ford Coppola',
#                                       'Rumble Fish is an American 1983 drama film directed by Francis Ford Coppola.')

# input_text = ('Give me all movies directed by Francis Ford Coppola ENTITY/Francis_Ford_Coppola',
#                                     'Rumble Fish ENTITY/Rumble_Fish  is an American ENTITY/United_States  1983 drama film directed by Francis Ford Coppola ENTITY/Francis_Ford_Coppola .')# It is based on the novel Rumble Fish ENTITY/Rumble_Fish  by S. E. Hinton ENTITY/S._E._Hinton , who also co-wrote the screenplay.The film centers on the relationship between Motorcycle Boy (Mickey Rourke ENTITY/Mickey_Rourke ), a revered former gang leader wishing to live a more peaceful life, and his younger brother, Rusty James ENTITY/Rumble_Fish  (Matt Dillon ENTITY/Matt_Dillon ), an uncool teenaged hoodlum who aspires to become as feared as Motorcycle Boy.')

# Query = 'Show me all songs from Bruce Springsteen ENTITY/Bruce_Springsteen  released between 1980 and 1990.'
# Passage = "Point Blank ENTITY/Sean_Kingston  is a song written by Bruce Springsteen ENTITY/Bruce_Springsteen  and first released on Springsteen ENTITY/Bruce_Springsteen 's 1980 album The River ENTITY/The_River_(Bruce_Springsteen_song) .  In Europe ENTITY/Europe , it was also released as a single in 1981, backed by another song from The River ENTITY/The_River_(Bruce_Springsteen_song) , Ramrod ENTITY/Super_Troopers   Although it was not released as a single in the US ENTITY/Billboard_Hot_100 , it did reach #20 on the Billboard Mainstream Rock Tracks ENTITY/Mainstream_Rock_(chart)  chart."

Query = 'Give me the capitals of all countries in Africa ENTITY/Africa'
Passage = 'Dakar ENTITY/Dakar  is the capital and largest city of Senegal'# .It is located on the Cap-Vert Peninsula ENTITY/Cap-Vert  on the Atlantic coast ENTITY/East_Coast_of_the_United_States  and is the westernmost city in the Old World ENTITY/Europe  and on the African ENTITY/Africa  mainland.'

# Query = 'Give me the capitals of all countries in Africa'
# Passage = 'List of African  dependencies — including the respective capitals.'

#Query = 'Give me all movies directed by Francis Ford Coppola ENTITY/Francis_Ford_Coppola'
#Passage = 'Rumble Fish ENTITY/Rumble_Fish  is an American ENTITY/United_States  1983 drama film directed by Francis Ford Coppola ENTITY/Francis_Ford_Coppola. It is based on the novel Rumble Fish ENTITY/Rumble_Fish  by S. E. Hinton ENTITY/S._E._Hinton , who also co-wrote the screenplay.The film centers on the relationship between Motorcycle Boy (Mickey Rourke ENTITY/Mickey_Rourke ), a revered former gang leader wishing to live a more peaceful life, and his younger brother, Rusty James ENTITY/Rumble_Fish  (Matt Dillon ENTITY/Matt_Dillon ), an uncool teenaged hoodlum who aspires to become as feared as Motorcycle Boy.'

#Passage = 'Christopher Coppola ENTITY/Christopher_Coppola  (born January 25, 1962) is a film director and producer.'


# Query = 'Show me all songs from Bruce Springsteen released between 1980 and 1990.'
# Passage = "Bruce Springsteen 's Video Anthology / 1978–88 is a collection of 18 music videos made on his behalf, released in VHS format on January 31, 1989.It was reissued as The Complete Video Anthology / 1978-2000 by Sony in DVD on January 16, 2001, adding a second disc with 15 additional music videos or other clips."

input_text = (Query, Passage)

# input_text = ('Give me all movies directed by Francis Ford Coppola',
#                                      'Christopher Coppola (born January 25, 1962) is a film director and producer')

# input_text = ('Give me all movies directed by Francis Ford Coppola ENTITY/Francis_Ford_Coppola',
#                                     'Christopher Coppola ENTITY/Christopher_Coppola  (born January 25, 1962) is a film director and producer.')


#input_text = ('What is the capital of Tuvalu ENTITY/Tuvalu', 'It is Funafuti ENTITY/Funafuti')



In [14]:
ret = reranker.tokenize_with_entities(*input_text, concat = True)


#ret_ref = make_reference(ret['input_ids'])
input_ids = reranker.vectorize(ret['input_ids'], model_emb, mapper, wiki_emb).to(device)
#ref_input_ids = reranker.vectorize(ret_ref, model_emb, mapper, wiki_emb).to(device)
token_type_ids =  ret['token_type_ids'].to(device)
#ref_token_type_ids = ref_token_ids(token_type_ids).to(device)

#ret['input_ids'], ret['token_type_ids']

all_tokens = ret['input_ids']
print(all_tokens)

['[CLS]', 'give', 'me', 'the', 'capitals', 'of', 'all', 'countries', 'in', 'africa', '/', 'ENTITY/Africa', 'africa', '[SEP]', 'da', '##kar', '/', 'ENTITY/Dakar', 'da', '##kar', 'is', 'the', 'capital', 'and', 'largest', 'city', 'of', 'senegal', '[SEP]']


In [16]:
ret = reranker.tokenize_with_entities(*input_text)

#ret = reranker.tokenize_with_entities('What is the capital city of France', 'It is Paris')


#ret_ref = make_reference(ret['input_ids'])
input_ids = reranker.vectorize(ret['input_ids'], model_emb, mapper, wiki_emb).to(device)
#ref_input_ids = reranker.vectorize(ret_ref, model_emb, mapper, wiki_emb).to(device)
token_type_ids =  ret['token_type_ids'].to(device)
#ref_token_type_ids = ref_token_ids(token_type_ids).to(device)

#ret['input_ids'], ret['token_type_ids']

all_tokens = ret['input_ids']
print(all_tokens)

['[CLS]', 'give', 'me', 'the', 'capitals', 'of', 'all', 'countries', 'in', 'africa', '/', 'ENTITY/Africa', '[SEP]', 'da', '##kar', '/', 'ENTITY/Dakar', 'is', 'the', 'capital', 'and', 'largest', 'city', 'of', 'senegal', '[SEP]']


In [18]:
# Load model and retrieve attention
#model_version = 'bert-base-uncased'
#do_lower_case = True
#model = BertModel.from_pretrained(model_version, output_attentions=True)
#tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)
#sentence_a = "The cat sat on the mat"
#sentence_b = "The cat lay on the rug"
#inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
#token_type_ids = inputs['token_type_ids']
#input_ids = inputs['input_ids']
attention = model(inputs_embeds = input_ids.float(), token_type_ids=token_type_ids.long())[-1]
#input_id_list = input_ids[0].tolist() # Batch index 0
#tokens = tokenizer.convert_ids_to_tokens(input_id_list)

In [19]:
len(all_tokens)

26

In [20]:
attention

(tensor([[[-0.0771, -0.2276, -0.2115,  ..., -0.1406, -0.2277,  0.1183],
          [-0.6424,  0.1819, -0.7668,  ...,  0.9381, -0.7975,  0.1105],
          [ 0.5036, -0.5936,  1.0275,  ...,  0.5699,  1.2150,  0.4628],
          ...,
          [ 0.5882,  0.3434, -0.0348,  ..., -0.0466,  0.3253,  0.5850],
          [ 1.0370, -0.2095, -0.1305,  ...,  0.4277,  0.3670, -0.0204],
          [-0.2332,  0.2289, -0.0506,  ...,  0.2634,  0.3046,  0.2425]]],
        device='cuda:0', grad_fn=<NativeLayerNormBackward>),
 tensor([[[-0.0551, -0.0922, -0.1385,  ...,  0.0046, -0.0835, -0.0605],
          [-0.9527,  0.1549, -0.4586,  ...,  0.9570, -0.9576,  0.2355],
          [ 0.5960, -0.9410,  0.9340,  ...,  0.5513,  1.7566,  0.3449],
          ...,
          [ 0.7757,  0.3955,  0.0337,  ..., -0.0783,  0.0973,  0.7050],
          [ 1.0743,  0.0236,  0.1310,  ...,  0.2392,  0.4489, -0.1576],
          [-0.1849,  0.2025, -0.1439,  ...,  0.2068,  0.3029,  0.3541]]],
        device='cuda:0', grad_fn=<NativeL

In [21]:
import numpy as np
sub_attention = [np.round(100*a.item()) for a in attention[0][0][0]]

In [87]:
#sub_attention

In [88]:
#help(head_view)

In [89]:
#list(zip(sub_attention, all_tokens))

In [90]:
#from TextAttentionHeatmapVisualization import text_attention

In [91]:
#pathname = 'text_attention_latex/africa.tex'
#text_attention.generate(all_tokens, sub_attention, pathname, color='green')

In [22]:
head_view(attention, all_tokens, layer=0, heads=[0])

ValueError: The attention tensor does not have the correct number of dimensions. Make sure you set output_attentions=True when initializing your model.